# Kaggle Competition 

In [1]:
import numpy as np
import pandas as pd
import polars as pl
import joblib

from sklearn.metrics import r2_score


# Load train/val data

In [2]:
train_data = pd.read_parquet("data/train_data.parquet")

In [3]:
train_data.shape

(29220048, 101)

In [4]:
val_data = pd.read_parquet("data/validate_data.parquet")

In [5]:
val_data.shape

(1082224, 101)

# define parameter

In [6]:
# feature selection ## select every feature except 0-4 
feature_cols = ["symbol_id", "time_id"] + [f"feature_{idx:02d}" for idx in range(79)]+ [f"responder_{idx}_lag_1" for idx in range(9)]
# feature_cols = [f"feature_{idx:02d}" for idx in range(79)]+ [f"responder_{idx}_lag_1" for idx in range(9)]

#select target values
target_cols = ["responder_6"]

# select the weight values
weight_cols = ["weight"]

# data scaling

In [7]:
X_train =   train_data[feature_cols]
y_train =   train_data[target_cols]
w_train =   train_data[weight_cols]
del train_data

In [8]:
# the chunk requir maximum memory

X_val =     val_data[feature_cols]
y_val =     val_data[target_cols]
w_val =     val_data[weight_cols]

del  val_data # free memory

# import model

# Check model similarity

# model ensemble

## voitng c

In [9]:
from sklearn.ensemble import VotingRegressor

In [2]:
model1 = joblib.load("model/XGBoost_ver4.4.pkl")    #0070
model2 = joblib.load("model/XGBoost_ver4.1.pkl")    #0069
model3 = joblib.load("model/XGBoost_ver4.2.pkl")    #0068
model4 = joblib.load("model/XGBoost_ver3.1.pkl")    #0067
model5 = joblib.load("model/XGBoost_ver4.3.pkl")    #0066

In [3]:
model1.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 1.0,
 'device': 'cuda',
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': 0,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': 0.01,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 7,
 'max_leaves': None,
 'min_child_weight': 3,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 1000,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': 42,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 0.7,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [4]:
model2.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 0.8,
 'device': 'cuda',
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': 0.05,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 6,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 200,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': 42,
 'reg_alpha': 1,
 'reg_lambda': 5,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 0.8,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [5]:
model3.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 0.7,
 'device': 'cuda',
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': 0.05,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 7,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 200,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': 42,
 'reg_alpha': 0.1,
 'reg_lambda': 1,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 0.9,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [13]:
ensemble_model = VotingRegressor(estimators=[
    ('model1', model1),
    ('model2', model2),
    ('model3', model3),
    # ('model4', model4),
    # ('model5', model5),

],
verbose= True,
weights= [2,1,1]
)  # Use 'hard' for majority voting or 'soft' for weighted average probabilities

# Fit the ensemble model (assuming you have training data X_train and y_train)
ensemble_model.fit(X_train, y_train, sample_weight= w_train)

C:\Users\Asus\AppData\Roaming\Python\Python311\site-packages\sklearn\ensemble\_voting.py:694: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[Voting] ................... (1 of 3) Processing model1, total=39.5min
[Voting] ................... (2 of 3) Processing model2, total= 6.1min
[Voting] ................... (3 of 3) Processing model3, total= 5.5min


VotingRegressor(estimators=[('model1',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=1.0, device='cuda',
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=0, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rat...
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=0.05, max_bin=None,
                                          max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=7,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None, n_estimators=200,
                                          n_jobs=None, num_parallel_tree=None,
                                          random_state=42, ...))],
                verbose=True, weights=[2, 1, 1])

In [14]:
joblib.dump(ensemble_model, "model/XGBoost_voting_ensemble7.pkl") 

['model/XGBoost_voting_ensemble7.pkl']

In [21]:
pred1 = ensemble_model.predict(X_val)
r2_score(y_val, pred1, sample_weight=w_val)

0.007959026446818118

In [2]:
model  = joblib.load("model/XGBoost_voting_ensemble6.pkl")

NameError: name 'joblib' is not defined

In [22]:
pred2 = model.predict(X_val)
r2_score(y_val,pred2, sample_weight= w_val)

0.007910490036010742

In [1]:
import matplotlib.pyplot as plt
plt.scatter(pred1, pred2)

NameError: name 'pred1' is not defined